<a href="https://colab.research.google.com/github/aju22/Transformers/blob/main/Transformers_from_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [126]:
import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.keras.layers as layers
import numpy as np

In [127]:
class selfAttention(layers.Layer):
  def __init__(self, embed_size, heads):
    super().__init__()

    self.embed_size = embed_size
    self.heads = heads
    self.head_dim = self.embed_size // self.heads

    assert (self.head_dim*heads == embed_size), "Embed size needs to be divisible by heads"

    self.values = layers.Dense(self.head_dim, use_bias = False)
    self.keys = layers.Dense(self.head_dim, use_bias = False) 
    self.queries = layers.Dense(self.head_dim, use_bias = False)
    self.fc_out = layers.Dense(self.embed_size)

  def call(self, values, keys, query, mask):

    N = query.shape[0]
    values_len, key_len, query_len = values.shape[1], keys.shape[1], query.shape[1]

    values = tf.reshape(values, (N, values_len, self.heads, self.head_dim))
    keys = tf.reshape(keys, (N, key_len, self.heads, self.head_dim))
    queries = tf.reshape(query, (N, query_len, self.heads, self.head_dim))

    values = self.values(values)
    keys = self.keys(keys)
    queries = self.queries(queries)
    
    #query shape : (n, query_len, heads, heads_dim)
    #keys shape : (n, key_len, heads, heads_dim)
    #einsum uses einstein notation to do a batch wise multiplication
    #energy shape : (n, heads, query_len, key_len)
    energy = tf.einsum("nqhd,nkhd->nhqk", queries, keys)

    if mask is not None:
      energy = tf.where(mask == 0.0, float("-1e20"), energy) 

    attention =  tf.keras.activations.softmax((energy/(self.embed_size**0.5)), axis=-1)
    #attention shape : (n, heads, query_len, key_len)
    #values shape = (n, value_len, heads, heads_dim)
    #einsum shape = (n, query_len, heads, heads_dim)
    out = tf.einsum("nhql,nlhd->nqhd", attention, values)
    out = tf.reshape(out, (N, query_len, self.heads*self.head_dim))  

    return self.fc_out(out)

In [128]:
class TransformerBlock(layers.Layer):
  def __init__(self, embed_size, heads, dropout, forward_expansion):
    super().__init__()

    self.attention = selfAttention(embed_size, heads)
    self.norm1 = layers.LayerNormalization()
    self.norm2 = layers.LayerNormalization()

    self.feed_forward = tf.keras.Sequential([
                                           
        layers.Dense(forward_expansion*embed_size, activation = 'relu'),
        layers.Dense(embed_size)
    ])

    self.dropout = layers.Dropout(dropout)


  def call(self, value, key, query, mask):

    attention = self.attention(value, key, query, mask)

    x = self.norm1(attention + query)
    
    x = self.dropout(x)
    
    forward = self.feed_forward(x)
    
    out = self.dropout(self.norm2(forward + x))

    return out

In [129]:
class Encoder(layers.Layer):
  def __init__(self, src_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, max_length):

    super().__init__()

    self.embed_size = embed_size
    self.word_embedding = layers.Embedding(src_vocab_size, embed_size)
    self.position_embedding = layers.Embedding(max_length, embed_size)

    self.layerlist = [TransformerBlock(embed_size, heads, dropout, forward_expansion) for _ in range(num_layers)]
    self.dropout = layers.Dropout(dropout)


  def call(self, x, mask):
    N, seq_length = x.shape

    positions = tf.cast(tf.tile(tf.expand_dims(tf.range(0, seq_length), axis = 0), [N, 1]), tf.float32)

    out = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

    for layer in self.layerlist:
      out = layer(out, out, out, mask)

    return out  

In [130]:
class DecoderBlock(layers.Layer):
  def __init__(self, embed_size, heads, forward_expansion, dropout):

    super().__init__()
    self.attention = selfAttention(embed_size, heads)
    self.norm = layers.LayerNormalization()
    self.transformer_block = TransformerBlock(embed_size, heads, dropout, forward_expansion)
    self.dropout = layers.Dropout(dropout)

  def call(self, x, value, key, src_mask, trg_mask):

    attention = self.attention(x, x, x, trg_mask)  
    query = self.dropout(self.norm(attention + x))
    out = self.transformer_block(value, key, query, src_mask)
    return out  

In [131]:
class Decoder(layers.Layer):
  def __init__(self, trg_vocab_size, embed_size, num_layers, heads, forward_expansion, dropout, max_length):

    super().__init__()
    self.embed_size = embed_size
    self.word_embedding = layers.Embedding(trg_vocab_size, embed_size)
    self.position_embedding = layers.Embedding(max_length, embed_size)
    self.layerlist = [DecoderBlock(embed_size, heads, forward_expansion, dropout) for _ in range(num_layers)]
    self.fc_out = layers.Dense(trg_vocab_size)
    self.dropout = layers.Dropout(dropout)
  
  def call(self, x , enc_out, src_mask, trg_mask):
    N, seq_length = x.shape

    positions = tf.cast(tf.tile(tf.expand_dims(tf.range(0, seq_length), axis = 0), [N, 1]), tf.float32)
    x = self.dropout(self.word_embedding(x) + self.position_embedding(positions))

    for layer in self.layerlist:
      x = layer(x, enc_out, enc_out, src_mask, trg_mask)

    out = self.fc_out(x)

    return out

In [132]:
class Transformer(layers.Layer):

  def __init__(self, 
               src_vocab_size, 
               trg_vocab_size,
               src_pad_idx,
               trg_pad_idx,
               embed_size = 256,
               num_layers = 6,
               forward_expansion = 4,
               heads = 8,
               dropout = 0,
               max_length = 100
               ):
    
    super().__init__()

    self.encoder = Encoder(src_vocab_size = src_vocab_size , embed_size = embed_size, num_layers = num_layers, 
                           heads = heads, forward_expansion = forward_expansion, dropout = dropout, max_length = max_length)
    self.decoder = Decoder(trg_vocab_size = trg_vocab_size , embed_size = embed_size, num_layers = num_layers, 
                           heads = heads, forward_expansion = forward_expansion, dropout = dropout, max_length = max_length)

    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx

  def make_src_mask(self, src):
    src_mask = tf.expand_dims(tf.expand_dims((src != self.src_pad_idx), 1), 2)  
    #(N, 1, 1, src_len)
    src_mask = tf.cast(src_mask, dtype = float)
    return src_mask

  def make_trg_mask(self, trg):

     N, trg_len = trg.shape

     trg_mask = tfp.math.fill_triangular(tf.ones((trg_len*(trg_len+1)//2, )), upper=False)
     trg_mask = tf.tile(tf.expand_dims(tf.expand_dims(trg_mask, axis = 0), 1), [N, 1, 1, 1])
     
     return trg_mask

  def call(self, src, trg):
    
    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)
    enc_src = self.encoder(src, src_mask)
    out = self.decoder(trg, enc_src, src_mask, trg_mask)

    return out

In [133]:
class Transformer(layers.Layer):

  def __init__(self, 
               src_vocab_size, 
               trg_vocab_size,
               src_pad_idx,
               trg_pad_idx,
               embed_size = 256,
               num_layers = 6,
               forward_expansion = 4,
               heads = 8,
               dropout = 0,
               max_length = 100
               ):
    
    super().__init__()

    self.encoder = Encoder(src_vocab_size = src_vocab_size , embed_size = embed_size, num_layers = num_layers, 
                           heads = heads, forward_expansion = forward_expansion, dropout = dropout, max_length = max_length)
    self.decoder = Decoder(trg_vocab_size = trg_vocab_size , embed_size = embed_size, num_layers = num_layers, 
                           heads = heads, forward_expansion = forward_expansion, dropout = dropout, max_length = max_length)

    self.src_pad_idx = src_pad_idx
    self.trg_pad_idx = trg_pad_idx

  def make_src_mask(self, src):
    src_mask = tf.expand_dims(tf.expand_dims((src != self.src_pad_idx), 1), 2)  
    #(N, 1, 1, src_len)
    src_mask = tf.cast(src_mask, dtype = float)
    return src_mask

  def make_trg_mask(self, trg):

     N, trg_len = trg.shape

     trg_mask = tfp.math.fill_triangular(tf.ones((trg_len*(trg_len+1)//2, )), upper=False)
     trg_mask = tf.tile(tf.expand_dims(tf.expand_dims(trg_mask, axis = 0), 1), [N, 1, 1, 1])
     
     return trg_mask

  def call(self, src, trg):
    
    src_mask = self.make_src_mask(src)
    trg_mask = self.make_trg_mask(trg)
    enc_src = self.encoder(src, src_mask)
    out = self.decoder(trg, enc_src, src_mask, trg_mask)

    return out

In [134]:
x = np.array([[1,5,6,4,3,9,5,2,0], 
              [1,8,7,3,4,5,6,7,2]
              ] , dtype = np.float32)
y = np.array([[1,7,4,3,5,9,2,0], 
              [1,5,6,2,4,7,6,2]
              ], dtype = np.float32)

src_pad_idx = 0
trg_pad_idx = 0
src_vocab_size = 10
trg_vocab_size = 10

model = Transformer(src_vocab_size, trg_vocab_size, src_pad_idx, trg_pad_idx)

In [135]:
out = model(x, y[:,:-1])

In [137]:
out.shape

TensorShape([2, 7, 10])